# 作为服务供小型团队使用

小型团队指那种使用人数不高于10人且规模基本固定的团队.这种团队一般是小公司研发团队.架设jupyterhub一般是为了共享工作站的算力资源,同时可以比较轻易的分享成果.

这种规模的小团队通常共享一台工作站,因此也就不需要什么集群化,docker单机模式就可以搞定.docker部署的最大优势就是可以在提供统一环境的同事隔离每个用户的私人环境.

部署的例子我放在[项目]()下

小型团队的特点是人少,并发小而稳定,工作场所固定通常只需要内网部署且希望尽可能的减少对外部的依赖.这边提供的部署方案是:

+ 使用docker部署单机hub
+ 使用sqlite作为数据库
+ 使用docker部署SPAWNER单用户服务
+ 使用账户密码注册登录服务


这里用到了三样组件

+ [hub服务](https://jupyterhub.readthedocs.io/en/latest/reference/api/app.html)
+ [dockerspawner](https://jupyterhub-dockerspawner.readthedocs.io/en/latest/index.html)
+ [jupyterhub-nativeauthenticator](https://native-authenticator.readthedocs.io/en/latest/index.html)

## 使用docker部署单机hub


jupyter官方提供了镜像[jupyterhub/jupyterhub](https://hub.docker.com/r/jupyterhub/jupyterhub)([Dockerfile](https://github.com/jupyterhub/jupyterhub/blob/main/Dockerfile))用于使用容器技术部署hub.

我们要实现上面的部署方案需要额外安装对应的依赖.

这里我已经基于`4.0.1`版本构造了镜像[hsz1273327/jupyterhub:small-4.0.1]().其构造dockerfile如下:


In [7]:
# %load examples/smallteam/hub/dockerfile
FROM --platform=$TARGETPLATFORM jupyterhub/jupyterhub:4.0.1 as base
RUN pip install jupyterhub-nativeauthenticator
RUN pip install dockerspawner
COPY jupyterhub_config.py /srv/jupyterhub/jupyterhub_config.py
CMD ["jupyterhub", "-f", "/srv/jupyterhub/jupyterhub_config.py"]

这个版本的镜像支持在环境变量中设置大部分实用的配置项.

可以使用环境变量设置的项目包括

> HUB设置

+ HUB_CONNECT_IP: 默认`jupyterhub`,连接hub可以使用的ip
+ HUB_PORT: 默认`8000`,指定hub对外的端口,注意8001和8081都不能使用,
+ HUB_SSL_KEY和HUB_SSL_CERT: 可选,用于设置https方式提供服务,需要都提供
+ HUB_COOKIE_SECRET_FILE: 默认`/data/jupyterhub_cookie_secret`,hub的cookie_secret文件存放位置,注意指的是容器中的存放位置
+ HUB_DB_URL: 默认`sqlite:////data/jupyterhub.sqlite`,hub数据保存的数据库位置
+ HUB_ACTIVE_SERVER_LIMIT: 默认100,指定挂载spawner服务的最大数量,超过这个数量后hub服务将不会启动新的spawner服务,通常一个用户绑定一个spawner服务
+ HUB_ACTIVE_USER_WINDOW: 默认1800,活跃用户总数的检查间隔时间,单位s
+ HUB_ACTIVITY_RESOLUTION: 默认30,活跃用户状态刷新间隔时间,单位s,这个参数用于限制状态写入的频率
+ HUB_CONCURRENT_SPAWN_LIMIT: 默认20,同时并发的spawn数量最大值
+ HUB_INTERNAL_SSL_PATH: 可选,指定内部通信启用ssl时ssl文件存放的位置,如果设置则启用内部ssl通信
+ HUB_INIT_SPAWNERS_TIMEOUT: 默认`60`,spawner初始化最大时长

> SPAWNER设置

+ SPAWNER_NOTEBOOK_IMAGE: 默认值`jupyter/base-notebook:notebook-6.5.4`,指定SPAWNER拉起作为notebook执行器的容器使用的镜像
+ SPAWNER_PULL_POLICY: 默认值`ifnotpresent`,枚举类型,指定拉取NOTEBOOK_IMAGE使用的策略,可选的有:
    + `ifnotpresent`-如果没有就拉取
    + `always`-总是检查更新并拉取
    + `never`- 总是不拉取,如果本地没有就抛出错误
    + `skip`- 总是不拉取,但也不抛出错误
+ SPAWNER_NETWORK_NAME: 必填,指定使用的docker network名
+ SPAWNER_NOTEBOOK_DIR: 默认`/home/jovyan/work`,指定notebook容器中notebook存放路径,不建议修改
+ SPAWNER_CPU_LIMIT: 默认`2`,指定notebook容器最大可使用的cpu资源量
+ SPAWNER_CPU_GUARANTEE: 默认`1`,指定notebook容器最低使用的cpu资源量
+ SPAWNER_MEM_LIMIT: 默认`4G`,指定notebook容器最大可使用的内存资源量
+ SPAWNER_MEM_GUARANTEE: 默认`1G`,指定notebook容器最小可使用的内存资源量
+ SPAWNER_ENVIRONMENT: 选填,使用`A:1;B:2`这样的形式指定notebook容器的环境变量
+ SPAWNER_SPAWN_CMD: 默认`start-singleuser.sh`,指定notebook容器的启动执行命令
+ SPAWNER_REMOVE: 默认`True`,Spawner容器在程序停止后是否删除容器,注意为true时没有保存在对应volume中的数据会丢失
+ SPAWNER_DEBUG: 默认`True`,Spawner容器设为debug模式
+ SPAWNER_CONSECUTIVE_FAILURE_LIMIT: 默认0,Spawner关闭与hub连接前允许的最大故障数,0表示不限制
+ SPAWNER_POLL_INTERVAL: 默认30, 轮询Spawner的间隔,单位s
+ SPAWNER_START_TIMEOUT: 默认120,单用户容器启动最大等待时间,单位s
+ SPAWNER_USE_GPUS: 选填,spawner对应的容器是否需要使用gpu,使用几个gpu,可以为正整数或为-1或者all

> AUTH设置

+ AUTH_ADMIN_USER: 默认`admin`,指定admin用户的用户名,多个用`,`隔开
+ AUTH_CHECK_COMMON_PASSWORD: 默认`False`,是否启用复杂密码校验
+ AUTH_MINIMUM_PASSWORD_LENGTH: 默认`6`,密码长度校验
+ AUTH_ALLOWED_FAILED_LOGINS: 默认`3`,最大登录错误次数
+ AUTH_SECONDS_BEFORE_NEXT_TRY: 默认`1200`,重试最长间隔时间
+ AUTH_ENABLE_SIGNUP: 默认`True`,是否开放新用户注册
+ AUTH_OPEN_SIGNUP: 默认`False`,是否允许用户注册后未经管理员确认就可以进入使用

部署的时候推荐使用`docker-compose`方式部署.

在部署前我们需要先构造一个可以让新建的容器可以挂在的网络,假定命名为`jupyterhub_network`,其创建命令如下:

```bash
docker network create --driver bridge --attachable jupyterhub_network
```

然后创建`docker-compose.yml`:

```docker-compose
version: "2.4"
services:
    jupyterhub:
        build: .
        # This is necessary to prevent the singleton hub from using its service number as its hostname
        hostname: jupyterhub
        container_name: jupyterhub
        # Permit communication with the host's docker server
        networks:
            - jupyterhub_network
        ports:
            - 8001:8000
        volumes:
            - "/var/run/docker.sock:/var/run/docker.sock"
            - "jupyterhub-data:/data"
        # Ensure Hub and Notebook servers are on the same network
        environment:
            HUB_PORT: 8000
            SPAWNER_NETWORK_NAME: jupyterhub_network

volumes:
    jupyterhub-data:

networks:
    jupyterhub_network:
        external: true
```

### 原生的用户管理

小团队内部使用自然没道理用外部的账号进行验证,因此我们安装`jupyterhub-nativeauthenticator`模块.这个模块的作用是提供一个本地的账户系统.

这部分的设置可以查看上面以`AUTH`开头的环境变量说明

#### 账户系统的工作流

`jupyterhub-nativeauthenticator`模块提供的账户系统注册流程包含两步:

1. 注册账号,这一步仅是创建一个用户,这个用户如果不是管理员用户默认并不能进入系统中使用jupyterhub提供的服务.
2. 验证账户,这一步需要有一个管理员账户来执行,在jupyterhub的`/hub/authorize`页面中验证已经注册的用户.非管理员用户只有被验证过才可以进入jupyterhub使用它提供的服务.


`jupyterhub-nativeauthenticator`模块提供的账户系统区分一般用户和管理员,管理员名字需要在配置文件中设置.如果你使用我提供的镜像,可以设置环境变量`AUTH_ADMIN_USER`来进行设置.

管理员账户天然是已经通过验证的账号了,因此可以直接使用jupyterhub提供的服务,同时管理员账户有验证注册账号的功能.

![本地账户系统的工作流](source/native_auth_flow.png)

如果你的团队太小觉得多一道管理员验证账户的步骤太麻烦,在使用我提供的镜像的前提下可以使用`AUTH_OPEN_SIGNUP: "True"`来关闭这一设置.


#### 账户系统的登录流程

`jupyterhub-nativeauthenticator`模块提供的账户系统对登录流程也有设置,默认会有3次尝试登录机会,当三次都登录失败后需要默认等待1200秒冷却,之后才能继续尝试

![本地账户系统登录重试流程](source/native_auth_process.webp)

## 使用容器隔离环境

在jupyterhub中每个用户实际使用的环境是一个单独的jupyterlab服务,而spawner提供了针对这个jupyterlab服务的注册和生命周期管理工作,官方提供的spawner有几个种类供选择,我们希望每个用户都在一个沙盒中运行自己的notebook,因此使用docker是一个相对比较靠谱的方案.我们只需要一个docker hub实例来进行分发,后面的每个spawner则管理一个单独的docker容器即可,每个用户一个容器,做到充分隔离.我们可以通过挂载`volume`的方式保存用户的notebook.这样即便容器崩了删了,只要用户名不变就还可以找回来.


需要注意spawner通过docker方式部署notebook服务会直接从镜像拉起运行容器,容器会根据设置的构造创建命令.需要注意如下几点:

+ 网络不好的留足初始化时间,如果使用我构造的镜像可以通过适当调大环境变量`HUB_INIT_SPAWNERS_TIMEOUT`和`SPAWNER_START_TIMEOUT`来实现
+ 每个用户都会创建一个命名形式为`jupyterhub-user-{username}`的volume来保存各自的notebook.
+ 使用我构造的镜像可以通过设置`SPAWNER_USE_GPUS`来让容器使用gpu.当然这需要配合支持gpu的镜像使用,
+ 一个jupyter hub系统只能指定特定的一个镜像.

### 选择单用户镜像

jupyter官方提供了许多选择,不过都是cpu版本的.下面是他们的关系图,我们可以在根据这张图选择需要的镜像名,然后去[dockerhub](https://hub.docker.com/u/jupyter)上确定合适的版本.

![官方镜像关系](source/notebook-relation.png)

如果希望容器可以使用gpu,则可以使用项目[GPU-Jupyter](https://github.com/iot-salzburg/gpu-jupyter)下的镜像,要注意gpu镜像会比较大(超过5g)且该项目只支持x86_64.它提供三种镜像:

+ 无后缀镜像,表示全功能,包含Julia和R支持.
+ `python-only`后缀镜像,表示只支持python核心
+ `slim`后缀镜像,也就是最小化镜像,只有python核心且无各种python库

推荐镜像[cschranz/gpu-jupyter:v1.5_cuda-11.6_ubuntu-20.04_python-only](https://hub.docker.com/r/cschranz/gpu-jupyter/tags),需要注意这个项目目前的cuda版本为11.6,目前pytorch最低要求cuda11.7,因此如果你希望用最新的torch建议自己构建.

### 自定义单用户镜像

如果以上镜像都无法满足我们的需求,比如我们希望默认带一些示例notebook,或者我们常用的库以上镜像中都没有,我们一样可以基于一个基镜像自己构造镜像.

推荐不需要gpu的可以[基于官方镜像构建](https://github.com/jupyter/docker-stacks).官方镜像基于`ubuntu`,使用`micromamba`管理依赖,当然也都有python和pip,可以很容易的进行定制,我们可以根据对应镜像的dockerfile来选择合适的基镜像进行进一步开发.比如我们有一个minio和一个postgresql存数据资源,我们需要安装可以访问他们的包以及pandas,matplotlib等可以方便查看数据的包,我们也希望修改pip和conda的,并提供一个默认的例子notebook,我们可以使用`jupyter/base-notebook`作为基镜像,安装对应工具并将例子文件复制进镜像

In [1]:
# %load examples/smallteam/cpubase-notebook/dockerfile
FROM --platform=$TARGETPLATFORM jupyter/base-notebook:notebook-6.5.4 as base
# 更换Ubuntu的源
USER root
RUN sed -i "s@http://.*archive.ubuntu.com@https://mirrors.tuna.tsinghua.edu.cn@g" /etc/apt/sources.list
RUN sed -i "s@http://.*security.ubuntu.com@https://mirrors.tuna.tsinghua.edu.cn@g" /etc/apt/sources.list
# 安装c++编译工具
RUN apt update -y && apt install -y --no-install-recommends \
    build-essential \
    ffmpeg \
    && rm -rf /var/lib/apt/lists/*

USER ${NB_UID}
# 更换conda的源
COPY --chown="${NB_UID}:${NB_GID}" initial-condarc "${CONDA_DIR}/.condarc"
# 更换pip源
COPY --chown="${NB_UID}:${NB_GID}" pip.conf "${HOME}/.pip/pip.conf"
# 安装需要的库
RUN conda install --channel conda-forge pygraphviz pydot lxml cython

RUN pip --no-cache-dir install --upgrade pip
RUN pip --no-cache-dir install ipywidgets
RUN pip --no-cache-dir install widgetsnbextension
RUN pip --no-cache-dir install boto3
RUN pip --no-cache-dir install psycopg2-binary
RUN pip --no-cache-dir install peewee
RUN pip --no-cache-dir install SQLAlchemy
RUN pip --no-cache-dir install matplotlib
RUN pip --no-cache-dir install seaborn

RUN pip --no-cache-dir install pandas
RUN pip --no-cache-dir install pyarrow
RUN pip --no-cache-dir install thefuzz
RUN pip --no-cache-dir install pyod
RUN pip --no-cache-dir install scipy
RUN pip --no-cache-dir install scikit-learn
RUN pip --no-cache-dir install scikit-image
RUN pip --no-cache-dir install skforecast
RUN pip --no-cache-dir install sympy
RUN pip --no-cache-dir install statsmodels
RUN pip --no-cache-dir install pingouin
RUN pip --no-cache-dir install mpmath
RUN pip --no-cache-dir install networkx
# 复制例子
COPY conntodata.ipynb /home/jovyan/work/examples/conntodata.ipynb

之后执行`docker buildx build --push --platform=linux/amd64,linux/arm64 -t {镜像名:标签} .`构造自定义镜像并保存到镜像仓库,这样就可以重复使用了.

需要gpu的情况则推荐基于英伟达的[官方基镜像](https://hub.docker.com/r/nvidia/cuda)构建.注意英伟达的官方镜像有三种标签:

+ 命名含有`base`的镜像,基础镜像,只包含cuda.
+ 命名含有`runtime`的镜像,适合作为运行时使用的镜像,除了cuda外还有nccl,cudnn等其他数学包.
+ 命名含有`devel`的镜像,适合作为多步构造中中间步骤使用的镜像,包含cuda相关的工具.

我们通常基于命名含有`runtime`的镜像来自定义镜像.这会相对复杂一些,我们需要参考jupyter官方的镜像构建过程.下面是两个基于官方镜像构造的镜像的dockerfile,可以拿去作为基镜像使用

+ `hsz1273327/docker-stacks-foundation-cuda:11.8.0-cudnn8-ubuntu22.04`

In [6]:
# %load examples/smallteam/gpubase-notebook/docker-stacks-foundation/dockerfile
FROM --platform=$TARGETPLATFORM nvidia/cuda:11.8.0-cudnn8-runtime-ubuntu22.04 as base

ARG NB_USER="jovyan"
ARG NB_UID="1000"
ARG NB_GID="100"

# Fix: https://github.com/hadolint/hadolint/wiki/DL4006
# Fix: https://github.com/koalaman/shellcheck/wiki/SC3014
SHELL ["/bin/bash", "-o", "pipefail", "-c"]

USER root

# Install all OS dependencies for notebook server that starts but lacks all
# features (e.g., download as all possible file formats)
ENV DEBIAN_FRONTEND noninteractive
RUN sed -i "s@http://.*archive.ubuntu.com@https://mirrors.tuna.tsinghua.edu.cn@g" /etc/apt/sources.list
RUN sed -i "s@http://.*security.ubuntu.com@https://mirrors.tuna.tsinghua.edu.cn@g" /etc/apt/sources.list
RUN apt-get update --yes && \
    # - apt-get upgrade is run to patch known vulnerabilities in apt-get packages as
    #   the ubuntu base image is rebuilt too seldom sometimes (less than once a month)
    apt-get upgrade --yes && \
    apt-get install --yes --no-install-recommends \
    # - bzip2 is necessary to extract the micromamba executable.
    bzip2 \
    ca-certificates \
    locales \
    sudo \
    # - tini is installed as a helpful container entrypoint that reaps zombie
    #   processes and such of the actual executable we want to start, see
    #   https://github.com/krallin/tini#why-tini for details.
    tini \
    wget && \
    apt-get clean && rm -rf /var/lib/apt/lists/* && \
    echo "en_US.UTF-8 UTF-8" > /etc/locale.gen && \
    locale-gen

# Configure environment
ENV CONDA_DIR=/opt/conda \
    SHELL=/bin/bash \
    NB_USER="${NB_USER}" \
    NB_UID=${NB_UID} \
    NB_GID=${NB_GID} \
    LC_ALL=en_US.UTF-8 \
    LANG=en_US.UTF-8 \
    LANGUAGE=en_US.UTF-8
ENV PATH="${CONDA_DIR}/bin:${PATH}" \
    HOME="/home/${NB_USER}"

# Copy a script that we will use to correct permissions after running certain commands
COPY fix-permissions /usr/local/bin/fix-permissions
RUN chmod a+rx /usr/local/bin/fix-permissions

# Enable prompt color in the skeleton .bashrc before creating the default NB_USER
# hadolint ignore=SC2016
RUN sed -i 's/^#force_color_prompt=yes/force_color_prompt=yes/' /etc/skel/.bashrc && \
   # Add call to conda init script see https://stackoverflow.com/a/58081608/4413446
   echo 'eval "$(command conda shell.bash hook 2> /dev/null)"' >> /etc/skel/.bashrc

# Create NB_USER with name jovyan user with UID=1000 and in the 'users' group
# and make sure these dirs are writable by the `users` group.
RUN echo "auth requisite pam_deny.so" >> /etc/pam.d/su && \
    sed -i.bak -e 's/^%admin/#%admin/' /etc/sudoers && \
    sed -i.bak -e 's/^%sudo/#%sudo/' /etc/sudoers && \
    useradd -l -m -s /bin/bash -N -u "${NB_UID}" "${NB_USER}" && \
    mkdir -p "${CONDA_DIR}" && \
    chown "${NB_USER}:${NB_GID}" "${CONDA_DIR}" && \
    chmod g+w /etc/passwd && \
    fix-permissions "${HOME}" && \
    fix-permissions "${CONDA_DIR}"

USER ${NB_UID}

# Pin python version here, or set it to "default"
ARG PYTHON_VERSION=3.11

# Setup work directory for backward-compatibility
RUN mkdir "/home/${NB_USER}/work" && \
    fix-permissions "/home/${NB_USER}"

# Download and install Micromamba, and initialize Conda prefix.
#   <https://github.com/mamba-org/mamba#micromamba>
#   Similar projects using Micromamba:
#     - Micromamba-Docker: <https://github.com/mamba-org/micromamba-docker>
#     - repo2docker: <https://github.com/jupyterhub/repo2docker>
# Install Python, Mamba and jupyter_core
# Cleanup temporary files and remove Micromamba
# Correct permissions
# Do all this in a single RUN command to avoid duplicating all of the
# files across image layers when the permissions change
COPY --chown="${NB_UID}:${NB_GID}" initial-condarc "${CONDA_DIR}/.condarc"
COPY --chown="${NB_UID}:${NB_GID}" pip.conf "${HOME}/.pip/pip.conf"
WORKDIR /tmp
RUN set -x && \
    arch=$(uname -m) && \
    if [ "${arch}" = "x86_64" ]; then \
        # Should be simpler, see <https://github.com/mamba-org/mamba/issues/1437>
        arch="64"; \
    fi && \
    wget --progress=dot:giga -O /tmp/micromamba.tar.bz2 \
        "https://micromamba.snakepit.net/api/micromamba/linux-${arch}/latest" && \
    tar -xvjf /tmp/micromamba.tar.bz2 --strip-components=1 bin/micromamba && \
    rm /tmp/micromamba.tar.bz2 && \
    PYTHON_SPECIFIER="python=${PYTHON_VERSION}" && \
    if [[ "${PYTHON_VERSION}" == "default" ]]; then PYTHON_SPECIFIER="python"; fi && \
    # Install the packages
    ./micromamba install \
        -c conda-forge \
        --root-prefix="${CONDA_DIR}" \
        --prefix="${CONDA_DIR}" \
        --yes \
        "${PYTHON_SPECIFIER}" \
        'mamba' \
        'jupyter_core' && \
    rm micromamba && \
    # Pin major.minor version of python
    mamba list python | grep '^python ' | tr -s ' ' | cut -d ' ' -f 1,2 >> "${CONDA_DIR}/conda-meta/pinned" && \
    mamba clean --all -f -y && \
    fix-permissions "${CONDA_DIR}" && \
    fix-permissions "/home/${NB_USER}"

# Configure container startup
ENTRYPOINT ["tini", "-g", "--"]
CMD ["start.sh"]

# Copy local files as late as possible to avoid cache busting
COPY start.sh /usr/local/bin/

# Switch back to jovyan to avoid accidental container runs as root
USER ${NB_UID}

WORKDIR "${HOME}"


+ `hsz1273327/base-notebook-cuda:notebook-6.5.4`

In [3]:
# %load examples/smallteam/gpubase-notebook/base-notebook/dockerfile
# Copyright (c) Jupyter Development Team.
# Distributed under the terms of the Modified BSD License.
ARG OWNER=hsz1273327
ARG BASE_CONTAINER=$OWNER/docker-stacks-foundation-cuda:latest
FROM --platform=$TARGETPLATFORM $BASE_CONTAINER

# Fix: https://github.com/hadolint/hadolint/wiki/DL4006
# Fix: https://github.com/koalaman/shellcheck/wiki/SC3014
SHELL ["/bin/bash", "-o", "pipefail", "-c"]

USER root

# Install all OS dependencies for notebook server that starts but lacks all
# features (e.g., download as all possible file formats)
RUN apt-get update --yes && \
    apt-get install --yes --no-install-recommends \
    build-essential \
    fonts-liberation \
    ffmpeg \
    git \
    # - pandoc is used to convert notebooks to html files
    #   it's not present in aarch64 ubuntu image, so we install it here
    pandoc \
    # - run-one - a wrapper script that runs no more
    #   than one unique  instance  of  some  command with a unique set of arguments,
    #   we use `run-one-constantly` to support `RESTARTABLE` option
    run-one && \
    apt-get clean && rm -rf /var/lib/apt/lists/*

USER ${NB_UID}

# Install Jupyter Notebook, Lab, and Hub
# Generate a notebook server config
# Cleanup temporary files
# Correct permissions
# Do all this in a single RUN command to avoid duplicating all of the
# files across image layers when the permissions change
WORKDIR /tmp
RUN mamba install --yes \
    'notebook' \
    'jupyterhub' \
    'jupyterlab' && \
    jupyter notebook --generate-config && \
    mamba clean --all -f -y && \
    npm cache clean --force && \
    jupyter lab clean && \
    rm -rf "/home/${NB_USER}/.cache/yarn" && \
    fix-permissions "${CONDA_DIR}" && \
    fix-permissions "/home/${NB_USER}"

ENV JUPYTER_PORT=8888
EXPOSE $JUPYTER_PORT

# Configure container startup
CMD ["start-notebook.sh"]

# Copy local files as late as possible to avoid cache busting
COPY start-notebook.sh start-singleuser.sh /usr/local/bin/
# Currently need to have both jupyter_notebook_config and jupyter_server_config to support classic and lab
COPY jupyter_server_config.py docker_healthcheck.py /etc/jupyter/

# Fix permissions on /etc/jupyter as root
USER root

# Legacy for Jupyter Notebook Server, see: [#1205](https://github.com/jupyter/docker-stacks/issues/1205)
RUN sed -re "s/c.ServerApp/c.NotebookApp/g" \
    /etc/jupyter/jupyter_server_config.py > /etc/jupyter/jupyter_notebook_config.py && \
    fix-permissions /etc/jupyter/

# HEALTHCHECK documentation: https://docs.docker.com/engine/reference/builder/#healthcheck
# This healtcheck works well for `lab`, `notebook`, `nbclassic`, `server` and `retro` jupyter commands
# https://github.com/jupyter/docker-stacks/issues/915#issuecomment-1068528799
HEALTHCHECK --interval=5s --timeout=3s --start-period=5s --retries=3 \
    CMD /etc/jupyter/docker_healthcheck.py || exit 1

# Switch back to jovyan to avoid accidental container runs as root
USER ${NB_UID}

WORKDIR "${HOME}"

下面则是安装了pytorch和tensorflow的镜像

+ `hsz1273327/torch-notebook-cuda:notebook6.5.4-torch2.0.1-cu118`

In [4]:
# %load examples/smallteam/gpubase-notebook/torch-notebook/dockerfile
ARG OWNER=hsz1273327
ARG BASE_CONTAINER=$OWNER/base-notebook-cuda:latest
FROM --platform=$TARGETPLATFORM $BASE_CONTAINER
COPY conntodata.ipynb /home/jovyan/work/examples/conntodata.ipynb
RUN pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118
RUN pip install boto3
RUN pip install psycopg2-binary
RUN pip install peewee
RUN pip install SQLAlchemy
RUN pip install matplotlib
RUN pip install pandas
RUN pip install pyarrow


+ `hsz1273327/tf-notebook-cuda:notebook6.5.4-tf2.12.0-cu118`

In [5]:
# %load examples/smallteam/gpubase-notebook/tf-notebook/dockerfile
ARG OWNER=hsz1273327
ARG BASE_CONTAINER=$OWNER/base-notebook-cuda:latest
FROM --platform=$TARGETPLATFORM $BASE_CONTAINER
COPY conntodata.ipynb /home/jovyan/work/examples/conntodata.ipynb
RUN manba install -c conda-forge cudatoolkit=11.8.0
RUN pip install nvidia-cudnn-cu11==8.6.0.163 tensorflow==2.12.*

RUN mkdir -p $CONDA_DIR/etc/conda/activate.d
RUN echo 'CUDNN_PATH=$(dirname $(python -c "import nvidia.cudnn;print(nvidia.cudnn.__file__)"))' >> $CONDA_DIR/etc/conda/activate.d/env_vars.sh
RUN echo 'export LD_LIBRARY_PATH=$LD_LIBRARY_PATH:$CONDA_DIR/lib/:$CUDNN_PATH/lib' >> $CONDA_DIR/etc/conda/activate.d/env_vars.sh
RUN source $CONDA_DIR/etc/conda/activate.d/env_vars.sh

RUN pip install boto3
RUN pip install psycopg2-binary
RUN pip install peewee
RUN pip install SQLAlchemy
RUN pip install matplotlib
RUN pip install pandas
RUN pip install pyarrow
